In [1]:
# from magicgame import *
import os
import random
import gym
from gym import Env
import numpy
from gym.spaces import Discrete
from enum import Enum
os.system('cls')

class Action(Enum):
    # Allowed Actions
    # u = untapped, t = tapped, c = creature, l = land, s = spell
    # s_u_c is to play a spell targeting an untapped creature
    play_c = 0
    play_s_u_c = 1
    play_s_t_c = 2
    play_s_self = 3
    play_s_opp = 4
    play_l = 5
    attack_c = 6
    block_u_c = 7
    pass_phase = 8

class Community_Data:
    # Data available to everyone
    def __init__(self):
        self.p1_life = 20
        self.p2_life = 20
        self.p1_deck = 60
        self.p2_deck = 60
        self.p1_num_hand = 0
        self.p2_num_hand = 0
        self.p1_battlefield = []
        self.p2_battlefield = []

class Player_Data:
    def __init__(self):
        self.position = 0
        self.deckconstruction = [0, 0, 0]

class Stage(Enum):
    main = 0
    attack = 1
    block = 2

class Card_Data(Enum):
    # u = untapped, t = tapped, c = creature, l = land, s = spell
    u_c = 0 
    s = 1
    u_l = 2
    t_c = 3
    t_l = 4
    
class magicgame(Env):
    def __init__(self, decksize = 60, num_creatures = 20, num_spells = 20):
        self.p1_life = 20
        self.p2_life = 20
        self.p1_decksize = decksize
        self.p2_decksize = decksize
        self.p1_battlefield = []
        self.p2_battlefield = []
        self.p1_deck = self.builddeck(decksize, num_creatures, num_spells)
        self.p2_deck = self.builddeck(decksize, num_creatures, num_spells)
        random.shuffle(self.p1_deck)
        random.shuffle(self.p2_deck)
        self.p1_hand = []
        self.p2_hand = []
        self.p1_draw(7)
        self.p2_draw(7)
        self.p1_landforturn = False
        self.p2_landforturn = False
        
        #phases for each player will be main phase 1, attacker phase, and blocking phase (1, 2, 3)
        self.phase = 1

    def builddeck(self, decksize, num_creatures, num_spells):
        deck = []
        for c in [0,1,2]:
            if c == 0:
                for i in range(0,num_creatures):
                    deck.append(0)
            elif c == 1:
                for i in range(0, num_spells):
                    deck.append(1)
            else:
                for i in range(0, decksize - num_creatures - num_spells):
                    deck.append(2)
        return deck

    def p1_draw(self, number_draw = 1):
        self.p1_landforturn = False
        for i in range(number_draw):
            self.p1_hand.append(self.p1_deck.pop(0))

    def p2_draw(self, number_draw = 1):
        self.p2_landforturn = False
        for i in range(number_draw):
            self.p2_hand.append(self.p2_deck.pop(0))

    def choose_card_choice(legalmoves):
        card_choice = np.random(legalmoves)
        return card_choice
        
    def p1_play(self, card_choice, target_choice = 0):
        if card_choice == 0: #creature
            manapool = 0
            for k in range(len(self.p1_battlefield)):
                if self.p1_battlefield[k] == 2:
                    # Check manacost of creature
                        if manapool < 2:
                            manapool +=1
            if manapool < 2:
                print("You dont have enough untapped mana to play that")
            
            elif card_choice in self.p1_hand: #check if the card is in the players hand
                tapper = 0
                for k in range(len(self.p1_battlefield)):
                    if self.p1_battlefield[k] == 2:
                        if tapper < 2:
                            self.p1_battlefield[k] = 4
                            tapper += 1
                self.p1_hand.remove(card_choice)
                self.p1_battlefield.append(0)
            else:
                print("You can't play that.")

        elif card_choice == 1: #spell
            # Play a spell, target choices are [p1, p2, p1 u_c, p1 t_c, p2 u_c, p2 t_C]
            # going from 1-6
            
            # First check to see if enough mana available to play spell
            manapool = 0
        
            for k in range(len(self.p1_battlefield)):
                if self.p1_battlefield[k] == 2:
                    if manapool < 1:
                        manapool += 1
            print("mana is ", manapool)
            if manapool < 1:
                print("You dont have enough untapped mana to play that")
            elif target_choice == 3 & 0 not in self.p1_battlefield: #check if the target does NOT exists
                print("Not a valid target")
            elif target_choice == 4 & 3 not in self.p1_battlefield: #check if the target does NOT exists
                print("Not a valid target")
            elif target_choice == 5 & 0 not in self.p2_battlefield: #check if the target does NOT exists
                print("Not a valid target")
            elif target_choice == 6 & 3 not in self.p2_battlefield: #check if the target does NOT exists
                print("Not a valid target")
            elif card_choice in self.p1_hand: #check if the card is in the players hand
                # Tap the appropriate mana
                tapper = 0
                for k in range(len(self.p1_battlefield)):
                    if self.p1_battlefield[k] == 2:
                        if tapper < 1:
                            self.p1_battlefield[k] = 4
                            tapper += 1
                # spell effects
                # Based on choice, remove that amount of life or that kind of creature from
                # appopriate battlefield

                if target_choice == 1:
                    self.p1_life = self.p1_life - 3
                    print("Player 1's life total was targeted by a spell.")
                elif target_choice == 2:
                    self.p2_life = self.p2_life - 3
                    print("Player 2's life total was targeted by a spell.")
                elif target_choice == 3:
                    if 0 in self.p1_battlefield: self.p1_battlefield.remove(0)
                    print("Player 1's untapped creature was targeted by a spell.")
                elif target_choice == 4:
                    if 3 in self.p1_battlefield: self.p1_battlefield.remove(3)
                    print("Player 1's tapped creature was targeted by a spell.")
                elif target_choice == 5:
                    if 0 in self.p2_battlefield: self.p2_battlefield.remove(0)
                    print("Player 2's untapped creature was targeted by a spell.")
                else:
                    if 3 in self.p2_battlefield: self.p2_battlefield.remove(3)
                    print("Player 2's tapped creature was targeted by a spell.")

                self.p1_hand.remove(card_choice)
            else:
                print("You can't play that.")


        elif card_choice == 2:
            
            if self.p1_landforturn == True:
                print("You have already played a land for turn")
                choose_card_choice([1])
            elif card_choice in self.p1_hand:
                self.p1_hand.remove(card_choice) #this needs to kick error, no land to play
                self.p1_battlefield.append(card_choice)
                self.landforturn = True
            else:
                print("You can't play that.")

    def p2_play(self, card_choice, target_choice = 0):
        if card_choice == 0:
            manapool = 0
            for k in range(len(self.p2_battlefield)):
                if self.p2_battlefield[k] == 2:
                    # Check manacost of creature
                        if manapool < 2:
                            manapool +=1
            if manapool < 2:
                print("You dont have enough untapped mana to play that")
            elif card_choice in self.p2_hand:
                tapper = 0
                for k in range(len(self.p2_battlefield)):
                    if self.p2_battlefield[k] == 2:
                        if tapper < 2:
                            self.p2_battlefield[k] = 4
                            tapper += 1
                self.p2_hand.remove(card_choice)
                self.p2_battlefield.append(0)
            else:
                print("You can't play that.")

        elif card_choice == 1:
            # Play a spell, target choices are [p1, p2, p1 u_c, p1 t_c, p2 u_c, p2 t_C]
            # going from 1-6
            
            # First check to see if enough mana available to play spell
            manapool = 0
            
            for k in range(len(self.p2_battlefield)):
                if self.p2_battlefield[k] == 2:
                    if manapool < 1:
                        self.p2_battlefield[k] == 4
                        manapool +=1

            if manapool < 1:
                print("You dont have enough untapped mana to play that")
            elif target_choice == 3 & 0 not in self.p1_battlefield: #check if the target does NOT exists
                print("Not a valid target")
            elif target_choice == 4 & 3 not in self.p1_battlefield: #check if the target does NOT exists
                print("Not a valid target")
            elif target_choice == 5 & 0 not in self.p2_battlefield: #check if the target does NOT exists
                print("Not a valid target")
            elif target_choice == 6 & 3 not in self.p2_battlefield: #check if the target does NOT exists
                print("Not a valid target")
            elif card_choice in self.p2_hand:
                # Tap the appropriate mana
                tapper = 0
                for k in range(len(self.p2_battlefield)):
                    if self.p2_battlefield[k] == 2:
                        if tapper < 1:
                            self.p2_battlefield[k] = 4
                            tapper += 1
                # Based on choice, remove that amount of life or that kind of creature from
                # appopriate battlefield

                if target_choice == 1:
                    self.p1_life = self.p1_life - 3
                    print("Player 1's life total was targeted by a spell.")
                elif target_choice == 2:
                    self.p2_life = self.p2_life - 3
                    print("Player 2's life total was targeted by a spell.")
                elif target_choice == 3:
                    if 0 in self.p1_battlefield: self.p1_battlefield.remove(0)
                    print("Player 1's untapped creature was targeted by a spell.")
                elif target_choice == 4:
                    if 3 in self.p1_battlefield: self.p1_battlefield.remove(3)
                    print("Player 1's tapped creature was targeted by a spell.")
                elif target_choice == 5:
                    if 0 in self.p2_battlefield: self.p2_battlefield.remove(0)
                    print("Player 2's untapped creature was targeted by a spell.")
                else:
                    if 3 in self.p2_battlefield: self.p2_battlefield.remove(3)
                    print("Player 2's tapped creature was targeted by a spell.")

                self.p2_hand.remove(card_choice)
            else:
                print("You can't play that.")

        elif card_choice == 2:
            self.p2_landforturn == False
            if self.p2_landforturn == True:
                print("You have already played a land for turn")
            elif card_choice in self.p2_hand:
                self.p2_hand.remove(card_choice)
                self.p2_battlefield.append(2)
                self.landforturn = True
            else:
                print("You can't play that.")
    
    def nextphase(self):
        self.phase += 1
        if self.phase == 4:
            self.phase = 11
        elif self.phase == 14:
            self.phase = 1
        else:
            pass

    def p1_attack(self, num_attackers):
        tapper = 0
        for k in range(len(self.p1_battlefield)):
            if self.p1_battlefield[k] == 0:
                if tapper < num_attackers:
                    self.p1_battlefield[k] = 3
                    tapper += 1
        self.phase += 1

    def p2_attack(self, num_attackers):
        tapper = 0
        for k in range(len(self.p2_battlefield)):
            if self.p2_battlefield[k] == 0:
                if tapper < num_attackers:
                    self.p2_battlefield[k] = 3
                    tapper += 1
        self.phase += 1

    def p1_block(self, num_block, num_attack):
        counter = 0
        for k in range(num_block):
            if k+1 <= num_attack:
                self.p2_battlefield.remove(3)
                self.p1_battlefield.remove(0)
            else:
                print("That's ok I guess")
        if num_attack >= num_block:
            damage = (num_attack - num_block) * 2
            self.p1_life -= damage
            print("Player 1 lost "+str(damage)+ " life.")
        else:
            print("overblocked")
        self.phase = 1

    def p2_block(self, num_block, num_attack):
        counter = 0
        for k in range(num_block):
            if k+1 <= num_attack:
                self.p1_battlefield.remove(3)
                self.p2_battlefield.remove(0)
            else:
                print("That's ok I guess")
        if num_attack >= num_block:
            damage = (num_attack - num_block) * 2
            self.p2_life -= damage
            print("Player 2 lost "+str(damage)+ " life.")
        else:
            print("overblocked")
        self.phase = 11

    def p1_turn(self):
        for k in range(len(self.p1_battlefield)):
            if self.p1_battlefield[k] == 3:
                    self.p1_battlefield[k] = 0
            if self.p1_battlefield[k] == 4:
                    self.p1_battlefield[k] = 2

    def p2_turn(self):
        for k in range(len(self.p2_battlefield)):
            if self.p2_battlefield[k] == 3:
                    self.p2_battlefield[k] = 0
            if self.p2_battlefield[k] == 4:
                    self.p2_battlefield[k] = 2

    def show_battlefield(self):
        print("\n __________________________________________________________")
        print("\n", self.p2_hand)
        print("\n _____ P2 _____ Life: ", self.p2_life)
        for k in range(len(self.p2_battlefield)):
            if self.p2_battlefield[k] == 2:
                print("L", end = " ")
            elif self.p2_battlefield[k] == 4:
                print("l", end = " ")
        print("\n")
        for k in range(len(self.p2_battlefield)):
            if self.p2_battlefield[k] == 0:
                print("C", end = " ")
            elif self.p2_battlefield[k] == 3:
                print("c", end = " ")
        print("\n")
        for k in range(len(self.p1_battlefield)):
            if self.p1_battlefield[k] == 0:
                print("C", end = " ")
            elif self.p1_battlefield[k] == 3:
                print("c", end = " ")
        print("\n")
        for k in range(len(self.p1_battlefield)):
            if self.p1_battlefield[k] == 2:
                print("L", end = " ")
            elif self.p1_battlefield[k] == 4:
                print("l", end = " ")
        print("\n _____ P1 _____ Life: ", self.p1_life)
        print("\n", self.p1_hand)
        
def chooseAction():
        values = [0,1,2,8]
        probs = [0.1,0.1,0.7,0.1]
        decision = np.random.choice(values, p = probs)
        return decision

# def reward():
    ## for use later
    ## +1 for valid turn, and -1 for invalid turn
    
game = magicgame()


In [2]:
def playgame():
    game = magicgame()
    p1_battlefield = []
    p2_battlefield = []
    while (game.p1_life > 0 and game.p2_life > 0) and (len(game.p2_deck) > 0):
        # p1 phase 1
        if game.phase == 1:
            # show the battle field
            game.show_battlefield()
            print("What does P1 want to play? 0 = creature, 1 = spell (and target of P1 = 1, P2 = 2, \
                P1 U creature = 3, P1 T creature = 4, P2 U creature = 5, P2 T creature = 6),\
                     2 = land, 8 = pass phase")
            
            p1_decision = chooseAction()
            p2_decision = 0

            # If player one plays a spell
            if p1_decision == 1:
                # Player one choose target
                print("Target?")
                values = [1,2,3,4,5,6]
                probs = [0.05,0.85/3,0.05,0.05,0.85/3,0.85/3]
                target = np.random.choice(values, p = probs) #P1 wants to target P2 stuff
                game.p1_play(p1_decision, target)
            elif p1_decision == 8:
                game.nextphase()
            else:
                game.p1_play(p1_decision)
        elif game.phase == 2:
            game.show_battlefield()
            print("How many P1 attackers? Any N attackers, 0 is pass")
            #global num_attack
            print(str(game.p1_battlefield.count(0)) + " p1 battlefield")
            values = list(range(0,game.p1_battlefield.count(0)))
            probs = [1/(game.p1_battlefield.count(0)+2)]*(game.p1_battlefield.count(0))
            probs.append(2/(game.p1_battlefield.count(0)+2))
            num_attack = np.random.choice(values, p = probs) # needs to be random
            print("Player 1 is attacking with " + str(num_attack) + " creatures.")
            game.p1_attack(num_attack)
        elif game.phase == 3:
            if num_attack == 0:
                game.nextphase()
                pass
            else:
                game.show_battlefield()
                print("How many P2 blockers? Any N blockers, 0 is pass")
                values = list(range(0,game.p2_battlefield.count(0)))
                probs = [1/(game.p2_battlefield.count(0)+2)]*(game.p2_battlefield.count(0))
                probs.append(2/(game.p2_battlefield.count(0)+2))
                p2_decision = np.random.choice(values,p = probs) # needs to be random
                print("Player 2 is blocking with " + str(p2_decision) + " creatures.")
                game.p2_block(p2_decision, num_attack)
            game.p2_turn()
            game.p2_draw()

        if game.phase == 11:
            game.show_battlefield()

            print("What does P2 want to play? 0 = creature, 1 = spell (and target of P1 = 1, P2 = 2, \
                P1 U creature = 3, P1 T creature = 4, P2 U creature = 5, P2 T creature = 6),\
                     2 = land, 8 = pass phase")

            p1_decision = 0
            p2_decision = chooseAction()

            if p2_decision == 1:
                print("Target?")
                target = random.randint(1,6) #P2 wants to target P1 stuff
                game.p2_play(p1_decision, target)
            elif p2_decision == 8:
                game.nextphase()
            else:
                game.p2_play(p2_decision)
        elif game.phase == 12:
            game.show_battlefield()
            print("How many P2 attackers? Any N attackers, 0 is pass")
            print(str(game.p2_battlefield.count(0)) + " p2 battlefield")
            num_attack = random.randint(0,game.p2_battlefield.count(0)) # needs to be random
            print("Player 2 is attacking with " + str(num_attack) + " creatures.")
            print(num_attack)
            game.p2_attack(num_attack)
        elif game.phase == 13:
            if num_attack == 0:
                game.nextphase()
                pass
            else:
                game.show_battlefield()
                print("How many P1 blockers? Any N blockers, 0 is pass" )
                p1_decision = random.randint(0,game.p1_battlefield.count(0)) # needs to be random
                print("Player 1 is blocking with " + str(p1_decision) + " creatures.")
                game.p1_block(p1_decision, num_attack)
            game.p1_turn()
            game.p1_draw()
        else:
            pass

# Winner / Loser
def show_winner():
    if a.p1_life < 1:
        print("P1 loses, P2 wins")
    elif a.p2_life < 1:
        print("P2 loses, P2 wins")
    elif len(a.p2_deck) < 0:
        print("P2 has no more cards in library, P1 wins")

In [ ]:
# for later use
def train(rounds=100):
    curr_round = 0
    while curr_round < rounds:
        play_game()
        curr_round += 1

 
def feedReward(self, reward):
        for st in reversed(self.states):
            if self.states_value.get(st) is None:
                self.states_value[st] = 0
            self.states_value[st] += self.lr * (self.decay_gamma * reward - self.states_value[st])
            reward = self.states_value[st]

In [ ]:
m1 = np.matrix([[[None,2],None,None], [None, None,None]])
m2 = np.matrix([[1,2,3],[2,3,4]])

m3 = np.concatenate((m1,m2))
m3

In [3]:
for k in range(20):
    print("Game: " + str(k+1))
    playgame()

Game: 1

 __________________________________________________________

 [2, 2, 2, 2, 0, 0, 1]

 _____ P2 _____ Life:  20







 _____ P1 _____ Life:  20

 [1, 0, 1, 2, 1, 1, 2]
What does P1 want to play? 0 = creature, 1 = spell (and target of P1 = 1, P2 = 2,                 P1 U creature = 3, P1 T creature = 4, P2 U creature = 5, P2 T creature = 6),                     2 = land, 8 = pass phase


NameError: name 'np' is not defined

In [ ]:
# Problem with Not a valid target error